In [1]:
# LIBRERÍAS
import os
import pandas as pd
from shutil import copyfile
from tqdm import tqdm

In [2]:
ruta_train_labels = "../data/train_labels.csv"
ruta_val_labels = "../data/val_labels.csv"
ruta_train_imagenes = "../data/train_set"
ruta_val_imagenes = "../data/val_set"
ruta_class_list = "../data/class_list.txt"
ruta_test_imagenes = "../data/test_set"

# CREAMOS LA ESTRUCTURA EN LA CARPETA DATA
os.makedirs("../data/images/train", exist_ok=True)
os.makedirs("../data/images/val", exist_ok=True)
os.makedirs("../data/labels/train", exist_ok=True)
os.makedirs("../data/labels/val", exist_ok=True)
os.makedirs("../data/images/test", exist_ok=True)
os.makedirs("../data/labels/test", exist_ok=True)

# CONFIGURAMOS EL TAMAÑO DE LOS BOUNDING BOXES
ancho_normalizado = 0.8  
alto_normalizado = 0.8   

In [3]:
train_labels = pd.read_csv(ruta_train_labels)
val_labels = pd.read_csv(ruta_val_labels)

In [4]:
from shutil import copyfile
import os
from tqdm import tqdm

# FUNCIÓN PARA PROCESAR IMÁGENES Y GENERAR BOUNDINGG BOXES
def procesar_imagenes_y_labels(labels_df, ruta_imagenes, salida_imagenes, salida_labels):
    for _, row in tqdm(labels_df.iterrows(), total=labels_df.shape[0], desc=f"Procesando {salida_imagenes}"):
        # Copiar la imagen a la carpeta YOLO
        imagen_origen = os.path.join(ruta_imagenes, row['img_name'])
        imagen_destino = os.path.join(salida_imagenes, row['img_name'])
        copyfile(imagen_origen, imagen_destino)

        # Generar anotación YOLO
        etiqueta = row['label']
        x_centro, y_centro = 0.5, 0.5  # Centro de la imagen
        bounding_box = f"{etiqueta} {x_centro} {y_centro} {ancho_normalizado} {alto_normalizado}\n"

        # Guardar el archivo .txt
        ruta_etiqueta = os.path.join(salida_labels, row['img_name'].replace('.jpg', '.txt'))
        with open(ruta_etiqueta, "w") as f:
            f.write(bounding_box)

# PROCESAMOS EL CONJUNTO DE ENTRENAMIENTO
procesar_imagenes_y_labels(
    train_labels,
    ruta_train_imagenes,
    "../data/images/train", 
    "../data/labels/train"  
)

# PROCESAMOS EL CONJUNTO DE VALIDACIÓN
procesar_imagenes_y_labels(
    val_labels,
    ruta_val_imagenes,
    "../data/images/val",   
    "../data/labels/val"   
)

# PROCESAR CONJUNTO DE TEST SIN LABELS
def procesar_imagenes_test(ruta_imagenes, salida_imagenes):
    for imagen in tqdm(os.listdir(ruta_imagenes), desc="Procesando imágenes de prueba"):
        imagen_origen = os.path.join(ruta_imagenes, imagen)
        imagen_destino = os.path.join(salida_imagenes, imagen)
        copyfile(imagen_origen, imagen_destino)

# CARPETA DE IMÁGNES DE TEST
os.makedirs("../data/images/test", exist_ok=True)

# PROCESAR IMÁGENES DE TEST
procesar_imagenes_test(
    "../data/test_set",     
    "../data/images/test"    
)

Procesando imágenes de prueba: 100%|████████████████████████████████████████████| 28377/28377 [02:56<00:00, 160.82it/s]


In [5]:
# LEER CLASES 
with open(ruta_class_list, "r") as f:
    clases = [line.strip().split(" ", 1)[1] for line in f.readlines()]  

# CREAMOS DATA.YAML
with open("../data/data.yaml", "w") as f:
    f.write(f"train: ../data/images/train\n")
    f.write(f"val: ../data/images/val\n")
    f.write(f"nc: {len(clases)}\n")  
    f.write(f"names: {clases}\n")  

print("Dataset preparado para YOLO.")

Dataset preparado para YOLO.


In [6]:
import shutil

# ELIMINAMOS CARPETAS Y ARCHIVOS INNECESARIOS
carpetas_a_eliminar = ["../data/train_set", "../data/val_set", "../data/test_set"]
archivos_a_eliminar = ["../data/train_labels.csv", "../data/val_labels.csv", "../data/test_set.zip", 
                       "../data/train_set.zip", "../data/val_set.zip", 
                       "../data/sample_submission.csv", "../data/ifood2019_sample_submission.csv"]


print("Inicio del proceso de eliminación de carpetas y archivos innecesarios...")

# Carpetas
for carpeta in carpetas_a_eliminar:
    if os.path.exists(carpeta):
        shutil.rmtree(carpeta)
        print(f"Carpeta eliminada: {carpeta}")
    else:
        print(f"Carpeta no encontrada (ya eliminada o no existe): {carpeta}")

# Archivos
for archivo in archivos_a_eliminar:
    if os.path.exists(archivo):
        os.remove(archivo)
        print(f"Archivo eliminado: {archivo}")
    else:
        print(f"Archivo no encontrado (ya eliminado o no existe): {archivo}")

print("Proceso de eliminación finalizado.")

Inicio del proceso de eliminación de carpetas y archivos innecesarios...
Carpeta eliminada: ../data/train_set
Carpeta eliminada: ../data/val_set
Carpeta eliminada: ../data/test_set
Archivo eliminado: ../data/train_labels.csv
Archivo eliminado: ../data/val_labels.csv
Archivo eliminado: ../data/test_set.zip
Archivo eliminado: ../data/train_set.zip
Archivo eliminado: ../data/val_set.zip
Archivo eliminado: ../data/sample_submission.csv
Archivo eliminado: ../data/ifood2019_sample_submission.csv
Proceso de eliminación finalizado.
